# Imports

In [1]:
import tensorflow as tf
import tqdm.keras
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow import keras
import numpy as np

# 1. Preparations

## 1.1 Load dataset



In [2]:
dataset = load_dataset('gnad10')

Found cached dataset gnad10 (C:/Users/tomas/.cache/huggingface/datasets/gnad10/default/1.1.0/3a8445be65795ad88270af4d797034c3d99f70f8352ca658c586faf1cf960881)


  0%|          | 0/2 [00:00<?, ?it/s]

## 1.2 Load BERT-Tokenizer and tokenize the data

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-german-cased')

## 1.3 Model architecture

In [4]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-german-cased', num_labels=9)
model.trainable = True
optimizer = keras.optimizers.Adam(learning_rate=1e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 1.4 Define the number of folds for cross validation

In [5]:
kf = KFold(n_splits=5, shuffle=True)

# 2. Fine tune the model on the available dataset using cross-validation

In [ ]:
for train_index, val_index in kf.split(dataset['train']):

    # Split train dataset into train and val datasets
    train_data = dataset['train'][train_index]
    val_data = dataset['train'][val_index]

    # Preprocess the data
    train_encodings = tokenizer(train_data['text'], truncation=True, padding=True, max_length=512)
    val_encodings = tokenizer(val_data['text'], truncation=True, padding=True, max_length=512)

    # Create tensorflow datasets, once for training and once for validation
    train_dataset = tf.data.Dataset.from_tensor_slices((
        dict(train_encodings),
        train_data['label']
    ))

    val_dataset = tf.data.Dataset.from_tensor_slices((
        dict(val_encodings),
        val_data['label']
    ))

    # Train the model
    model.fit(
        train_dataset.batch(16),
        epochs=3,
        validation_data=val_dataset.batch(16),
        verbose=1
    )

Epoch 1/3
